In [1]:
from lib.utilities import *
import matplotlib.pyplot as plt

%matplotlib widget

In [2]:
datadir = "." # All ZDA files in this directory + subdirectories are loaded
selected_filename = "Untitled004" # focus for subsequent analysis
file_type = '.tsm'
horizontal_cropping = [250, 750]  # This is the cropping recommended for 1024-px width
binning = 5

In [3]:
# Load data
processed = [] # to avoid re-processing later
data_loader = DataLoader()
if file_type == '.tsm':
    data_loader.load_all_tsm(data_dir=datadir + "/tsm_targets")

.\tsm_targets/Untitled004.tsm to be treated as TSM file to open
Reading file as 200 images of size 1024 x 320
TBN file designates origin as NI for this data.
Found 4 channels in BNC ratio: 1
{'points_per_trace': 200, 'raw_width': 1024, 'raw_height': 320, 'interval_between_samples': 1.0, 'number_of_trials': 1}
.\tsm_targets/Untitled005.tsm to be treated as TSM file to open
Reading file as 200 images of size 1024 x 320
TBN file designates origin as NI for this data.
Found 4 channels in BNC ratio: 1
{'points_per_trace': 200, 'raw_width': 1024, 'raw_height': 320, 'interval_between_samples': 1.0, 'number_of_trials': 1}


In [4]:
# Select data of interest        
selected_data = data_loader.select_data_by_keyword(selected_filename)

# Increase binning and cropping until PhotoZ can manage the image size
selected_data.clip_data(y_range=horizontal_cropping)
selected_data.bin_data(binning=binning)

raw_data, meta, rli = selected_data.get_data(), selected_data.get_meta(), selected_data.get_rli()

{'points_per_trace': 199, 'raw_width': 199, 'raw_height': 100, 'interval_between_samples': 1.0, 'number_of_trials': 1} (1, 199, 319, 500)


In [5]:
# Fill in missing metadata
meta['version'] = 5
meta['slice_number'] = 5
meta['location_number'] = 5
meta['record_number'] = 5
meta['camera_program'] = 5

meta['interval_between_trials'] = 5
meta['acquisition_gain'] = 5
meta['time_RecControl'] = 5

meta['reset_onset'] = 1
meta['reset_duration'] = 5
meta['shutter_onset'] = 5
meta['shutter_duration'] = 5

meta['stimulation1_onset'] = 1
meta['stimulation1_duration'] = 5
meta['stimulation2_onset'] = 5
meta['stimulation2_duration'] = 5

meta['acquisition_onset'] = 1
meta['interval_between_samples'] = 5

meta['raw_width'] = raw_data.shape[2]
meta['raw_height'] = raw_data.shape[3]
meta['points_per_trace'] = raw_data.shape[1]
meta['number_of_trials'] = raw_data.shape[0]

rli = {}
rli['rli_low'] = [0] * int(meta['raw_width'] * meta['raw_height'])
rli['rli_high'] = [0] * int(meta['raw_width'] * meta['raw_height'])
rli['rli_max'] = [0] * int(meta['raw_width'] * meta['raw_height'])

In [7]:
# Write data
print(raw_data)
zda_writer = ZDA_Writer()
zda_writer.write_zda_to_file(raw_data, meta, selected_filename + ".zda", rli)


[[[[ 287.8   294.08  293.68 ... 1068.92  951.76  865.88]
   [ 266.88  272.2   285.16 ... 1106.56 1000.72  888.8 ]
   [ 268.56  272.56  314.04 ... 1166.   1067.24  947.76]
   ...
   [ 266.16  263.44  263.4  ...  794.04  667.88  575.92]
   [ 266.52  262.2   262.64 ...  711.32  603.24  511.04]
   [ 213.84  212.04  211.52 ...  506.28  426.28  356.76]]

  [[ 283.56  287.84  289.24 ... 1064.6   951.88  860.52]
   [ 264.64  267.24  282.6  ... 1105.08  993.16  886.2 ]
   [ 264.08  271.08  309.48 ... 1156.4  1070.24  952.76]
   ...
   [ 260.88  258.04  258.48 ...  786.32  664.08  573.76]
   [ 263.88  260.76  261.76 ...  709.68  601.96  505.12]
   [ 211.28  211.72  212.   ...  502.88  425.24  354.72]]

  [[ 285.36  292.2   292.76 ... 1073.44  956.    861.4 ]
   [ 268.84  269.52  287.68 ... 1105.8   997.6   887.84]
   [ 266.96  273.72  314.8  ... 1164.64 1067.    949.96]
   ...
   [ 267.08  262.2   264.8  ...  784.68  673.32  576.8 ]
   [ 265.72  263.2   263.16 ...  709.4   603.6   505.24]
   [ 2